In [61]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    accuracy_score,
)

In [62]:
# issues with https
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [63]:
# Numpy deprecation issue
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

In [64]:
URL = "http://raw.githubusercontent.com/Aditya1001001/English-Premier-League/master/pos_modelling_data.csv"

In [65]:
df = pd.read_csv(URL)
df.head()

,Position,Clean sheets,Goals conceded,Tackles,Tackle success %,Blocked shots,Interceptions,Clearances,Recoveries,Successful 50/50s,...,Shots,Shooting accuracy %,Saves,Penalties saved,age,value_eur,overall,Arial Saves,Duels %,Aerial battles %
0,Midfielder,0.0,0.0,4.0,100,0.0,1.0,0.0,9.0,4.0,...,2.0,50,0.0,0.0,21,4400000,72,0.0,46.153846,25.000000
1,Defender,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,22,10500000,77,0.0,0.000000,0.000000
2,Forward,0.0,0.0,10.0,0,11.0,1.0,19.0,0.0,0.0,...,42.0,36,0.0,0.0,19,7500000,73,0.0,0.000000,0.000000
3,Midfielder,0.0,0.0,9.0,56,3.0,9.0,14.0,40.0,12.0,...,10.0,30,0.0,0.0,31,4800000,74,0.0,55.384615,58.333333
4,Midfielder,0.0,0.0,22.0,59,5.0,14.0,0.0,58.0,6.0,...,11.0,18,0.0,0.0,28,0,83,0.0,40.869565,36.666667


In [66]:
# Position is the target
X = df.drop('Position', axis=1)
y = df['Position']

In [67]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Perform Feature Selection using Boruta

In [68]:
# Define the Random Forest Classifer model
RFC = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=10)

In [69]:
# Pass the model to Boruta for feature evaluation
Boruta_Selector = BorutaPy(RFC, random_state=10, verbose=2)

In [70]:
# Fit the model
Boruta_Selector.fit(X_train.values, y_train.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	31
Tentative: 	3
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	31
Tentative: 	1
Rejected: 	2
Iteration: 	10 / 100
Confirmed: 	31
Tentative: 	1
Rejected: 	2
Iteration: 	11 / 100
Confirmed: 	31
Tentative: 	1
Rejected: 	2
Iteration: 	12 / 100
Confirmed: 	32
Tentative: 	0
Rejected: 	2


BorutaPy finished running.

Iteration: 	13 / 100
Confirmed: 	32
Tentative: 	0
Rejected: 	2


BorutaPy(estimator=RandomForestClassifier(max_depth=5, n_estimators=1000,
                                          random_state=RandomState(MT19937) at 0x1461A9A40),
         random_state=RandomState(MT19937) at 0x1461A9A40, verbose=2)

In [71]:
Boruta_Selector.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [72]:
Boruta_Selector.n_features_

32

32 of the 34 features were picked as significant, so we can drop the 2 insignificant features

In [73]:
# list(X_train.columns)

In [74]:
# Convert results into a dataframe with labels
Boruta_Selector_Ranked_Features = pd.DataFrame(
    {"FeatureName": list(X.columns), "Ranking": list(Boruta_Selector.ranking_), }
)

In [75]:
Boruta_Selector_Ranked_Features

,FeatureName,Ranking
0,Clean sheets,1
1,Goals conceded,1
2,Tackles,1
3,Tackle success %,1
4,Blocked shots,1
5,Interceptions,1
6,Clearances,1
7,Recoveries,1
8,Successful 50/50s,1
9,Own goals,2


- As an example, you get 70% accuracy before Boruta analysis and 72% after
- Even though accuracy only increased 2%, the overall performance gain of dropping two columns may still be significant